# Simple Chains and Prompt Templates - Interactive Notebook

This notebook provides hands-on examples for implementing chains and prompt templates in LangChain.

## 1. Setup and Imports

In [ ]:
# Import necessary libraries
import os
from dotenv import load_dotenv
import time
import json

# Load environment variables
load_dotenv()

# Core LangChain imports
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain_core.prompts.example_selector import LengthBasedExampleSelector
from langchain_core.chains import LLMChain, SequentialChain, TransformChain
from langchain_core.memory import ConversationBufferMemory

# Model imports
from langchain_openai import OpenAI, ChatOpenAI

print("Successfully imported LangChain components!")

## 2. Working with Prompt Templates

In [ ]:
# Create different types of prompt templates

# Simple template
simple_template = PromptTemplate(
    template="What is the capital of {country}?",
    input_variables=["country"]
)

# Multi-variable template
multi_var_template = PromptTemplate(
    template="Translate the following text from {source_lang} to {target_lang}: {text}",
    input_variables=["source_lang", "target_lang", "text"]
)

# Template with partial variables
partial_template = PromptTemplate(
    template="As a {role}, explain {topic} in a {tone} manner.",
    input_variables=["topic", "tone"],
    partial_variables={"role": "expert educator"}
)

print("Templates created successfully!")

# Format and print examples
print("Simple template:", simple_template.format(country="Japan"))
print("\nMulti-variable template:", multi_var_template.format(
    source_lang="English", 
    target_lang="Spanish", 
    text="Hello, how are you?"
))
print("\nPartial template:", partial_template.format(topic="quantum physics", tone="simple"))

## 3. Few-Shot Prompt Templates

In [ ]:
# Create few-shot examples
examples = [
    {"question": "What is 2+2?", "answer": "4"},
    {"question": "What is 5*3?", "answer": "15"},
    {"question": "What is 10-7?", "answer": "3"},
    {"question": "What is 8/2?", "answer": "4"},
    {"question": "What is 6+9?", "answer": "15"}
]

# Create example prompt
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\nAnswer: {answer}"
)

# Create few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples[:3],  # Use first 3 examples
    example_prompt=example_prompt,
    prefix="Here are some examples of math problems:",
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"],
    example_separator="\n\n"
)

print("Few-shot prompt template created!")
print("\nFormatted prompt:")
print(few_shot_prompt.format(input="What is 7*8?"))

## 4. Dynamic Example Selection

In [ ]:
# Create example selector for dynamic examples
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # Maximum length of examples
)

# Create dynamic prompt
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Here are some examples:",
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"],
    example_separator="\n\n"
)

print("Dynamic prompt with example selector created!")
print("\nFormatted dynamic prompt:")
print(dynamic_prompt.format(input="What is 12*15?"))

## 5. Chat Prompt Templates

In [ ]:
# Create chat prompt template
system_template = "You are a helpful assistant that specializes in {subject}."
human_template = "{user_input}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

# Format the chat prompt
formatted_messages = chat_prompt.format_messages(
    subject="history",
    user_input="Tell me about the Renaissance period."
)

print("Chat prompt template created!")
print("\nFormatted messages:")
for message in formatted_messages:
    print(f"{message.type}: {message.content}")

## 6. Simple LLM Chains

In [ ]:
# Initialize model
llm = OpenAI(temperature=0.7)

# Create a simple explanation chain
explanation_prompt = PromptTemplate(
    template="Explain {concept} in simple terms.",
    input_variables=["concept"]
)

explanation_chain = LLMChain(llm=llm, prompt=explanation_prompt, verbose=True)

# Test the chain
result = explanation_chain.run(concept="quantum computing")
print("\nExplanation:")
print(result)

## 7. Sequential Chains

In [ ]:
# Create multiple chains for sequential execution

# Chain 1: Generate a topic
topic_prompt = PromptTemplate(
    template="Generate an interesting topic about {subject}.",
    input_variables=["subject"]
)
topic_chain = LLMChain(llm=llm, prompt=topic_prompt, output_key="topic")

# Chain 2: Create content about the topic
content_prompt = PromptTemplate(
    template="Write a short paragraph about {topic}.",
    input_variables=["topic"]
)
content_chain = LLMChain(llm=llm, prompt=content_prompt, output_key="content")

# Chain 3: Summarize the content
summary_prompt = PromptTemplate(
    template="Summarize this content in one sentence: {content}",
    input_variables=["content"]
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")

# Combine into a sequential chain
overall_chain = SequentialChain(
    chains=[topic_chain, content_chain, summary_chain],
    input_variables=["subject"],
    output_variables=["topic", "content", "summary"],
    verbose=True
)

# Run the sequential chain
result = overall_chain({"subject": "artificial intelligence"})
print("\nSequential Chain Results:")
print(f"Topic: {result['topic']}")
print(f"Content: {result['content']}")
print(f"Summary: {result['summary']}")

## 8. Transform Chains

In [ ]:
# Create a transform chain to process input
def transform_func(inputs):
    """Transform input data"""
    text = inputs["text"]
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    
    return {
        "original_text": text,
        "word_count": word_count,
        "char_count": char_count,
        "analysis": f"The text has {word_count} words and {char_count} characters."
    }

transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["original_text", "word_count", "char_count", "analysis"],
    transform=transform_func
)

# Create a summary chain
summary_prompt = PromptTemplate(
    template="Analyze this text: {original_text}\n\n{analysis}\n\nProvide insights:",
    input_variables=["original_text", "analysis"]
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="insights")

# Combine chains
full_chain = SequentialChain(
    chains=[transform_chain, summary_chain],
    input_variables=["text"],
    output_variables=["analysis", "word_count", "char_count", "insights"]
)

# Test the transform + LLM chain
result = full_chain.run(text="LangChain is a powerful framework for building applications with large language models.")
print("\nTransform Chain Results:")
print(f"Analysis: {result['analysis']}")
print(f"Word Count: {result['word_count']}")
print(f"Character Count: {result['char_count']}")
print(f"Insights: {result['insights']}")

## 9. Chain with Memory Integration

In [ ]:
# Create a chain with integrated memory
memory = ConversationBufferMemory(memory_key="chat_history", input_key="input")

prompt_template = """
You are a helpful assistant. Here's your conversation history:

{chat_history}

Human: {input}
Assistant:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["input", "chat_history"]
)

memory_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

# Test the memory chain
response1 = memory_chain.run(input="Hi, I'm learning about LangChain.")
print(f"\nAssistant 1: {response1}")

response2 = memory_chain.run(input="Can you explain what chains are?")
print(f"\nAssistant 2: {response2}")

response3 = memory_chain.run(input="How do I create a sequential chain?")
print(f"\nAssistant 3: {response3}")

## 10. Exercise: Multi-Step Content Generator

In [ ]:
# TODO: Implement a multi-step content generator
# 1. Create prompts for each step
# 2. Create chains for each step
# 3. Combine them into a sequential chain
# 4. Test with different topics

# Step 1: Title generation
title_prompt = PromptTemplate(
    template="Generate a catchy blog post title about {topic}.",
    input_variables=["topic"]
)
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

# Step 2: Outline creation
outline_prompt = PromptTemplate(
    template="Create a 3-point outline for a blog post titled '{title}'.",
    input_variables=["title"]
)
outline_chain = LLMChain(llm=llm, prompt=outline_prompt, output_key="outline")

# Step 3: Introduction
intro_prompt = PromptTemplate(
    template="Write an engaging introduction for a blog post titled '{title}' with this outline: {outline}",
    input_variables=["title", "outline"]
)
intro_chain = LLMChain(llm=llm, prompt=intro_prompt, output_key="introduction")

# Step 4: Key points
key_points_prompt = PromptTemplate(
    template="Expand on these key points from the outline: {outline}",
    input_variables=["outline"]
)
key_points_chain = LLMChain(llm=llm, prompt=key_points_prompt, output_key="key_points")

# Step 5: Conclusion
conclusion_prompt = PromptTemplate(
    template="Write a strong conclusion for a blog post with this introduction: {introduction}",
    input_variables=["introduction"]
)
conclusion_chain = LLMChain(llm=llm, prompt=conclusion_prompt, output_key="conclusion")

# Combine all chains
content_generator = SequentialChain(
    chains=[title_chain, outline_chain, intro_chain, key_points_chain, conclusion_chain],
    input_variables=["topic"],
    output_variables=["title", "outline", "introduction", "key_points", "conclusion"],
    verbose=True
)

# Test the content generator
result = content_generator.run(topic="sustainable living")
print("\n=== CONTENT GENERATOR RESULTS ===")
print(f"Title: {result['title']}")
print(f"\nOutline: {result['outline']}")
print(f"\nIntroduction: {result['introduction']}")
print(f"\nKey Points: {result['key_points']}")
print(f"\nConclusion: {result['conclusion']}")

## 11. Summary

In this notebook, we've covered:
- Creating and customizing various types of prompt templates
- Implementing few-shot learning with examples
- Building simple and sequential chains
- Creating transform chains for data processing
- Integrating memory into chains
- Building a multi-step content generator

These techniques form the foundation for creating sophisticated LangChain applications that can handle complex workflows and maintain context across interactions.